In [1]:
import pandas as pd
import numpy as np
import PySAM.Singleowner as singleowner

from full_scrape import FullScrape
from base_processor import CRP_CHOICES
from extractor import Extractor

from tech_processors import OffShoreWindProc, LandBasedWindProc, DistributedWindProc,\
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,\
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,\
    CoalProc, NaturalGasProc, NuclearProc, BiopowerProc,\
    UtilityBatteryProc, CommBatteryProc, ResBatteryProc

from macrs import MACRS_6, MACRS_16, MACRS_21

In [2]:
# Data master version on sharepoint - empty string if you haven't renamed the file
version_string = "_v1.46"

# Path to data master spreadsheet
data_master_filename = '../2023-ATB-Data_Master' + version_string + '.xlsx'

techs = [
                OffShoreWindProc, LandBasedWindProc, DistributedWindProc,
                UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,
                CspProc, GeothermalProc, HydropowerProc, 
                CoalProc, NaturalGasProc, NuclearProc, BiopowerProc
            ]


CRP_CHOICES = [20]
scraper = FullScrape(data_master_filename, techs=techs)
scraper.scrape()

##### Processing OffShoreWind (1/14) #####
Loading data from Offshore Wind, for Market and 20
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from spreadsheet
Calculated LCOE matches LCOE from spreadsheet
Loading data from Offshore Wind, for R&D and 20
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from spreadsheet
Calculated LCOE matches LCOE from spreadsheet
Loading data from Offshore Wind, for Market and 30
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from spreadsheet
Calculated LCOE matches LCOE from spreadsheet
Loading data from Offshore Wind, for R&D and 30
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from spreadsheet
Calculated LCOE matches LCOE from spreadsheet
##### Processing LandbasedWind (2/14) #####
Loading data from Land-Based Wind, 

In [3]:
df_itc, df_ptc = Extractor.get_tax_credits_sheet(data_master_filename)

In [7]:
df_ptc.loc['Solar - Utility PV']

2021         0
2022       0.0
2023     25.46
2024     25.46
2025     25.46
2026     25.46
2027     25.46
2028     25.46
2029     25.46
2030     25.46
2031     25.46
2032     25.46
2033     25.46
2034     25.46
2035     25.46
2036     25.46
2037     25.46
2038     25.46
2039     25.46
2040    19.095
2041     12.73
2042       0.0
2043       0.0
2044       0.0
2045       0.0
2046       0.0
2047       0.0
2048       0.0
2049       0.0
2050       0.0
Name: Solar - Utility PV, dtype: object

In [22]:
def calculate_debt_fraction(input_vals, debug=False):
    model = singleowner.default("GenericSystemSingleOwner")

    analysis_period = 20
    ac_capacity = 1000 # kW
    capacity_factor = input_vals["CF"]
    gen = [capacity_factor * ac_capacity] * 8760 # Distribute evenly throughout the year

    capex = input_vals["OCC"]
    con_fin_costs = input_vals["CFC"]
    initial_investment = capex * ac_capacity
    con_fin_total = con_fin_costs * ac_capacity
    o_and_m = input_vals["Fixed O&M"]
    v_o_and_m = input_vals["Variable O&M"]
    dscr = input_vals["DSCR"]

    ## Set these here so we can adjust below
    irr_target = input_vals["IRR"] 
    tax_federal = input_vals["Tax Rate (Federal and State)"] * 100
    tax_state = 0
    inflation = input_vals["Inflation Rate"] * 100

    degradation = 0.0 # ATB presents average capactity factors

    model.value("analysis_period", analysis_period)
    model.value("flip_target_year", analysis_period)
    model.value("gen", gen)
    model.value("system_pre_curtailment_kwac", gen)
    model.value("system_capacity", ac_capacity)
    model.value("cp_system_nameplate", ac_capacity / 1000)
    model.value("total_installed_cost", initial_investment)

    ## Single Owner should apply the O&M cost to each year, so no need to multiply by analysis period?
    model.value("om_capacity", [o_and_m] ) 
    model.value("om_production", [v_o_and_m])

    model.value("degradation", [degradation]) # Specify length 1 so degradation is applied each year. An array of 0.7 len(analysis_period) assumes degradation the first year, but not afterwards
    model.value("system_use_lifetime_output", 0) # Do degradation in the financial model

    model.value("debt_option", 1) # Use DSCR
    model.value("dscr", dscr)
    # model.value("debt_percent", 51.9)
    model.value("inflation_rate", inflation)
    model.value("term_int_rate", input_vals['Interest Rate Nominal'] * 100)
    model.value("term_tenor", 18)
    model.value("real_discount_rate", input_vals['Calculated Rate of Return on Equity Real'] * 100) ## "real equity rate" (also get this from data?)
    model.value("flip_target_percent", irr_target) ## "nominal equity rate"
    model.value("ppa_escalation", 0.0)

    model.value("federal_tax_rate", [tax_federal])
    model.value("state_tax_rate", [tax_state])

    # This group is included in fixed O&M
    model.value("insurance_rate", 0)
    model.value("property_tax_rate", 0)
    model.value("prop_tax_cost_assessed_percent", 0)

    model.value("reserves_interest", 0)
    model.value("salvage_percentage", 0)
    model.value("months_receivables_reserve", 0)
    model.value("months_working_reserve", 0)
    model.value("dscr_reserve_months", 0)
    model.value("equip1_reserve_cost", 0)
    model.value("equip2_reserve_cost", 0)
    model.value("equip3_reserve_cost", 0)
    model.value("cost_debt_closing", 0)
    model.value("cost_debt_fee", 0)
    model.value("loan_moratorium", 0)
    model.value("construction_financing_cost", con_fin_total)
    model.value("itc_fed_percent", [input_vals["ITC"] * 100])
    model.value('itc_fed_percent_maxvalue', [1e38])
    model.value("itc_sta_amount", [0])
    model.value("ptc_fed_amount", [input_vals["PTC"] / 1000]) # Convert $/MWh to $/kWh

    if input_vals["MACRS"] == MACRS_6:
        model.value("depr_alloc_macrs_5_percent", 100)
        model.value("depr_itc_fed_macrs_5", 1)
        model.value("depr_itc_sta_macrs_5", 1)
        model.value("depr_alloc_macrs_15_percent", 0)
        model.value("depr_alloc_sl_20_percent", 0)
    elif input_vals["MACRS"] == MACRS_16:
        model.value("depr_alloc_macrs_5_percent", 0)
        model.value("depr_alloc_macrs_15_percent", 100)
        model.value("depr_itc_fed_macrs_15", 1)
        model.value("depr_itc_sta_macrs_15", 1)
        model.value("depr_alloc_sl_20_percent", 0)
    elif input_vals["MACRS"] == MACRS_21:
        model.value("depr_alloc_macrs_5_percent", 0)
        model.value("depr_alloc_macrs_15_percent", 0)
        model.value("depr_alloc_sl_20_percent", 100)
        model.value("depr_itc_fed_sl_20", 1)
        model.value("depr_itc_fed_sl_20", 1)
    model.value("depr_alloc_custom_percent", 0)
    model.value("depr_alloc_sl_5_percent", 0)
    model.value("depr_alloc_sl_15_percent", 0)
    model.value("depr_alloc_sl_39_percent", 0)
    model.value("depr_bonus_fed", 0)
    model.value("depr_bonus_sta", 0)
    model.value("depr_bonus_fed_macrs_5", 0)
    model.value("depr_bonus_sta_macrs_5", 0)
    model.value("depr_bonus_fed_macrs_15", 0)
    model.value("depr_bonus_sta_macrs_15", 0)

    model.value("depr_fedbas_method", 0)
    model.value("depr_stabas_method", 0)

    model.value("ppa_soln_mode", 0)
    model.value("payment_option", 0)

    model.value('en_electricity_rates', 1 )

    model.execute()

    if debug:
        print("LCOE: " + str(model.Outputs.lcoe_real)) #Cents / kWh - multiply by 10 to get $ / MWh
        print("NPV: " + str(model.Outputs.project_return_aftertax_npv))
        print()
        print("IRR in target year: " + str(model.Outputs.flip_target_irr))
        print("IRR at end of project: " + str(model.Outputs.analysis_period_irr))
        print("O&M: " + str(model.Outputs.cf_om_capacity_expense))
        print("PPA price: " + str(model.Outputs.cf_ppa_price))
        print("Debt Principal: " + str(model.Outputs.cf_debt_payment_principal))
        print("Debt Interest: " + str(model.Outputs.cf_debt_payment_interest))
        print("Depreciation: " + str(model.Outputs.cf_feddepr_total))
        print("Production: " + str(model.Outputs.cf_energy_net))
        print("Tax " + str(model.Outputs.cf_fedtax))
        print("ITC " + str(model.Outputs.itc_total_fed))
        print("PTC " + str(model.Outputs.cf_ptc_fed))
        print("Debt fraction " + str(model.Outputs.debt_fraction))

    return model.Outputs.debt_fraction

In [6]:
tech = UtilityPvProc
fin_case = 'Market'
year = 2021

d = scraper.data
print(d.columns)
input_vals = d[(d.DisplayName == tech.default_tech_detail) & (d.Case == fin_case) & (d.Scenario == 'Moderate') & (d.CRPYears == 20) & 
    ((d.Parameter == 'Fixed O&M') | (d.Parameter == 'Variable O&M') |(d.Parameter == 'OCC') | (d.Parameter == 'CFC') | (d.Parameter == 'CF'))]
input_vals = input_vals.set_index('Parameter')[year].to_dict()
print(input_vals)

Index([  'Parameter',        'Case',    'CRPYears',  'Technology',
       'DisplayName',    'Scenario',        2021.0,        2022.0,
              2023.0,        2024.0,        2025.0,        2026.0,
              2027.0,        2028.0,        2029.0,        2030.0,
              2031.0,        2032.0,        2033.0,        2034.0,
              2035.0,        2036.0,        2037.0,        2038.0,
              2039.0,        2040.0,        2041.0,        2042.0,
              2043.0,        2044.0,        2045.0,        2046.0,
              2047.0,        2048.0,        2049.0,        2050.0],
      dtype='object')
{'CF': 0.2462873695982513, 'OCC': 1139.2, 'Fixed O&M': 20.5568, 'Variable O&M': 0, 'CFC': 31.12886830479868}


In [18]:
d = scraper.data
print(set(d.Parameter))
gen_vals = d[(d.Technology == tech.tech_name) & (d.CRPYears == 20) & (d.Case == fin_case) & 
    ((d.Parameter == 'Inflation Rate') | (d.Parameter == 'Tax Rate (Federal and State)') | (d.Parameter == 'Calculated Rate of Return on Equity Real') | (d.Parameter == 'Interest Rate Nominal'))]
gen_vals = gen_vals.set_index('Parameter')[year].to_dict()
print(gen_vals)


{'Fixed O&M', 'Fuel', 'Heat Rate', 'CRC', 'Interest During Construction - Nominal', 'CAPEX', 'OCC', 'FCR', 'WACC Nominal', 'Variable O&M', 'CFC', 'LCOE', 'Tax Rate (Federal and State)', 'GCC', 'CF', 'CRF', 'WACC Real', 'Rate of Return on Equity Nominal', 'Calculated Rate of Return on Equity Real', 'Calculated Interest Rate Real', 'Interest Rate Nominal', 'Debt Fraction', 'Inflation Rate'}
{'Inflation Rate': 0.0279, 'Interest Rate Nominal': 0.05, 'Calculated Rate of Return on Equity Real': 0.04825372117910298, 'Tax Rate (Federal and State)': 0.25739999999999996}


In [24]:

if tech.has_itc and tech.has_ptc and fin_case == 'Market':
    input_vals["PTC"] = df_ptc.loc[tech.sheet_name][year]
    input_vals["ITC"] = df_itc.loc[tech.sheet_name][year]
else:
    input_vals["PTC"] = 0
    input_vals["ITC"] = 0
input_vals["DSCR"] = tech.dscr
input_vals["IRR"] = tech.irr_target
if isinstance(tech.depreciation_schedule, list):
    input_vals["MACRS"] = tech.depreciation_schedule
elif isinstance(tech.depreciation_schedule, dict):
    input_vals["MACRS"] = tech.depreciation_schedule[year]
input_vals.update(gen_vals)
print(input_vals)

{'CF': 0.2462873695982513, 'OCC': 1139.2, 'Fixed O&M': 20.5568, 'Variable O&M': 0, 'CFC': 31.12886830479868, 'PTC': 0, 'ITC': 0.30000001192092896, 'DSCR': 1.3, 'IRR': 7.75, 'MACRS': [0.2, 0.32, 0.192, 0.1152, 0.1152, 0.0576], 'Inflation Rate': 0.0279, 'Tax Rate (Federal and State)': 0.25739999999999996, 'Interest Rate Nominal': 0.05, 'Calculated Rate of Return on Equity Real': 0.04825372117910298}


In [23]:
print(input_vals)
debt_frac = calculate_debt_fraction(input_vals)
print(debt_frac)

{'CF': 0.2462873695982513, 'OCC': 1139.2, 'Fixed O&M': 20.5568, 'Variable O&M': 0, 'CFC': 31.12886830479868, 'PTC': 0, 'ITC': 0.30000001192092896, 'DSCR': 1.3, 'IRR': 7.75, 'MACRS': [0.2, 0.32, 0.192, 0.1152, 0.1152, 0.0576], 'Inflation Rate': 0.0279, 'Tax Rate (Federal and State)': 0.25739999999999996, 'Interest Rate Nominal': 0.05, 'Calculated Rate of Return on Equity Real': 0.04825372117910298}
49.8914771621317
